# Using OpenAI models with Strands Agent

## Overview

Strands Agents is an SDK that takes a model-driven approach to building and running AI agents in just a few lines of code. Strands supports multiple providers and models hosted anywhere.

[LiteLLM](https://docs.litellm.ai/docs/) is a unified interface for various LLM providers that allows you to interact with models from Amazon, Anthropic, OpenAI, and many others through a single API. The Strands Agent SDK implements a LiteLLM provider, allowing you to run agents against any model LiteLLM supports.

In this example, we will show you how to use `gpt-4.1-mini` model hosted in Microsoft Azure as the underlying model in your Strands Agent. We will use a simple agent use case with a weather and a get time tool.

## Agent Details

<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Feature used        |LiteLLM model                                      |
|Agent Structure     |Single agent architecture                          |

</div>


## Architecture

<div style="text-align:center">
    <img src="images/architecture.png" width="65%" />
</div>

## Key Features

- **LiteLLM model**: using a model provided via LiteLLM


## Setup and prerequisites

### Prerequisites

- Python 3.10+
- Azure Account
- gpt-4.1-mini access

Let's now install the requirement packages for our Strands Agent


In [1]:
# installing pre-requisites
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 110.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.5/725.5 kB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.5/352.5 kB 28.8 MB/s eta 0:00:00
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 4.19.2
    Uninstalling jsonschema-4.19.2:
      Successfully uninstalled jsonschema-4.19.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker 2.234.0 requires protobuf<5.0,>=3.12, but you have protobuf 5.29.5 which is incompatible.
sagemaker-core 1.0.14 requires rich<14.0.0,>=13.0.0, but you have rich 14.0.0 which is incompatible.


### Importing dependency packages

Now let's import the dependency packages


In [4]:
import os
from datetime import datetime
from datetime import timezone as tz
from typing import Any
from zoneinfo import ZoneInfo

from strands import Agent, tool
from strands.models.litellm import LiteLLMModel

### Setting up Azure keys

Let's now setup the Azure API Keys


In [ ]:
os.environ["AZURE_API_KEY"] = "<YOUR_API_KEY>"
os.environ["AZURE_API_BASE"] = "<YOUR_API_BASE>"
os.environ["AZURE_API_VERSION"] = "<YOUR_API_VERSION>"

### Setting up custom tools

Let's now setup two dummy tools to test our agent


In [9]:
@tool
def current_time(timezone: str = "UTC") -> str:
    if timezone.upper() == "UTC":
        timezone_obj: Any = tz.utc
    else:
        timezone_obj = ZoneInfo(timezone)

    return datetime.now(timezone_obj).isoformat()


@tool
def current_weather(city: str) -> str:
    # Dummy implementation. Please replace with actual weather API call.
    return "sunny"

### Defining agent underlying LLM model

Next let's define our agent underlying model using LiteLLM. We will set it to `gpt-4.1-mini`


In [ ]:
from strands.models.litellm import LiteLLMModel

# Azure OpenAI API用
openai_model = LiteLLMModel(
    model_id="azure/gpt-4.1-mini",  # 適切なモデルを選択
    params={
        "max_tokens": 4096,  # モデルの機能に基づいて調整
        "temperature": 0.7,  # より決定論的な応答には低く、より創造的には高く
        "top_p": 0.9,  # Nucleus samplingパラメータ
        "stream": True,  # ストリーミング応答を有効化
    },
)

### Defining Agent

Now that we have all the required information available, let's define our agent


In [ ]:
from strands_tools import current_time


@tool
def current_weather(city: str) -> str:
    """都市の現在の天気を取得します（ダミー実装）。"""
    return "晴れ"


# ツールを持つエージェントを作成
assistant = Agent(
    system_prompt="あなたは役に立つ AI アシスタントです。時間と天気についての質問にだけ答えてくれます。",  # システムプロンプトを定義
    model=openai_model,  # 先ほど設定したOpenAIモデルを使用
    tools=[current_time, current_weather],
)

### Testing agent

Let's now invoke the agent to test it


In [12]:
# ツールの使用が必要な質問をエージェントに尋ねる
result = assistant("シアトルは今何時ですか？天気はどうですか？")

# 会話履歴を確認できます
print(assistant.messages)

# 使用メトリクスも確認できます
print(result.metrics)


Tool #1: current_time

Tool #2: current_weather
シアトルの現在の時刻は午前6時43分です。天気は晴れています。ほかに知りたいことはありますか？[{'role': 'user', 'content': [{'text': 'シアトルは今何時ですか？天気はどうですか？'}]}, {'role': 'assistant', 'content': [{'toolUse': {'toolUseId': 'call_69EfuIY1WlBa8GwlQVF55QOM', 'name': 'current_time', 'input': {'timezone': 'US/Pacific'}}}, {'toolUse': {'toolUseId': 'call_f7FGuXCnVTmbXRK4xypU79BL', 'name': 'current_weather', 'input': {'city': 'Seattle'}}}]}, {'role': 'user', 'content': [{'toolResult': {'toolUseId': 'call_f7FGuXCnVTmbXRK4xypU79BL', 'status': 'success', 'content': [{'text': '晴れ'}]}}, {'toolResult': {'toolUseId': 'call_69EfuIY1WlBa8GwlQVF55QOM', 'status': 'success', 'content': [{'text': '2025-06-06T06:43:36.610368-07:00'}]}}]}, {'role': 'assistant', 'content': [{'text': 'シアトルの現在の時刻は午前6時43分です。天気は晴れています。ほかに知りたいことはありますか？'}]}]
EventLoopMetrics(cycle_count=2, tool_metrics={'current_weather': ToolMetrics(tool={'toolUseId': 'call_f7FGuXCnVTmbXRK4xypU79BL', 'name': 'current_weather', 'input': {'city': '

#### Analysing the agent's results

Nice! We've invoked our agent for the first time! Let's now explore the results object. First thing we can see is the messages being exchanged by the agent in the agent's object


In [ ]:
agent.messages

Next we can take a look at the usage of our agent for the last query by analysing the result `metrics`


In [ ]:
results.metrics

### Congratulations!

In this notebook you learned how to connect with MCP servers using Strands Agent and two MCP transport protocols: stdio and Streamable HTTP. You also learned how to connect multiple MCP servers to the same agent. Next, let's see how to use different models with your agent
